In [65]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import re
import time
import os
from selenium import webdriver

In [20]:
def get_soup(url):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) \
                AppleWebKit/537.36 (KHTML, like Gecko) \
                Chrome/58.0.3029.110 Safari/537.36'}
    
    response = requests.get(url, headers=headers)
    results = response.content
    if not response.status_code == 404:
        soup = BeautifulSoup(results, 'lxml')
    return soup

In [21]:
url_main = 'https://www.elliman.com/search/for-sale?sid=44458208'
apt_urls = get_soup(url_main).find_all('li', class_='listing_address first')

In [22]:
apt_url1 = 'https://www.elliman.com'+apt_urls[0].find('a')['href']
apt_url1

'https://www.elliman.com/new-york-city/12-east-63rd-street-manhattan-qjnryze'

In [70]:
from fake_useragent import UserAgent
ua = UserAgent()

In [15]:
def random_user_agent():
    try:
        ua = UserAgent()
        return ua.random
    except:
        default_ua = 'Mozilla/5.0 (Macintosh; Intel Mac O21S X 10_12_3) \
                AppleWebKit/537.36 (KHTML, like Gecko) \
                Chrome/58.0.3029.110 Safari/537.36'
        return default_ua

In [80]:
def get_soup(url):
    
    headers = {'User-Agent': random_user_agent()}
    
    response = requests.get(url, headers=headers)
    results = response.content
    if not response.status_code == 404:
        soup = BeautifulSoup(results, 'lxml')
    return soup

In [37]:
url = 'https://www.elliman.com/search/for-sale/search-1?sdid=1&sid=44458208&sk=1'
soup = get_soup(url)

In [38]:
apartments = soup.find_all('li', class_='listing_address first')
apt_urls = [apt.find('a')['href'] for apt in apartments]
apt_urls

['/new-york-city/12-east-63rd-street-manhattan-qjnryze',
 '/new-york-city/118-east-76th-street-manhattan-basrlqe',
 '/new-york-city/10-east-76th-street-manhattan-iuqifyo',
 '/new-york-city/11-st-lukes-place-manhattan-exnpepx',
 '/new-york-city/12-east-80th-street-manhattan-djuusxd',
 '/new-york-city/39-east-72nd-street-penthouse-manhattan-dwlnahy',
 '/new-york-city/18-east-74th-street-manhattan-gyioxmw',
 '/new-york-city/42-west-12th-street-manhattan-wokkpnq',
 '/new-york-city/48-east-81st-street-manhattan-vmjymzz',
 '/new-york-city/60-bethune-street-manhattan-syrzhui',
 '/new-york-city/29-west-75th-street-manhattan-bsebich',
 '/new-york-city/114-east-65th-street-manhattan-haugxns',
 '/new-york-city/20-east-94th-street-manhattan-lzxowfq',
 '/new-york-city/678-682-prospect-place-brooklyn-ebqvkmd',
 '/new-york-city/3-collister-street-manhattan-skolihu',
 '/new-york-city/345-lexington-avenue-manhattan-parstfj',
 '/new-york-city/210-first-avenue-manhattan-bwxdvpi',
 '/new-york-city/81-pier

In [45]:
apt_test = apt_urls[0]
soup_apt = get_soup('https://www.elliman.com/'+apt_test)
photo_link = soup_apt.find('li', class_='listing_all_photos').find('a')['href']
photo_link = 'https://www.elliman.com/'+photo_link
soup_photo = get_soup(photo_link)

In [62]:
imgs = soup_photo.find('div', class_='w_listitem_left').find_all('img')
imgs = ['https://www.elliman.com'+img['src'] for img in imgs]
imgs[0]

'https://www.elliman.com/img/bea5f2fa12a7efbafe19338afda41f149c333469+440++1'

In [78]:
def save_images(img_urls, data_path, address):
    try:
        current_path = os.getcwd()
        os.chdir(data_path)
        if not os.path.exists('Sale'):
            os.mkdir('Sale')
        os.chdir('Sale')
        
        if not os.path.exists(address):
            os.mkdir(address)
        os.chdir(address)
        
        for i, img_url in enumerate(img_urls):
            img_data = requests.get(img_url).content
            with open(f'img{i}.jpg', 'wb') as handler:
                handler.write(img_data)
                
        os.chdir(current_path)
        return 1
    except:
        return 0

In [79]:
save_images(imgs, '../data/sample/elliman/imgdata', 'test')

1

In [72]:
def get_webpage(pg_num):
    template = f'https://www.elliman.com/search/for-sale/search-{pg_num}?sdid=1&sid=44458208&sk=1'
    return template 

In [74]:
page1 = get_webpage(1)
page1

'https://www.elliman.com/search/for-sale/search-1?sdid=1&sid=44458208&sk=1'

In [81]:
soup_pg1 = get_soup(page1)
soup_pg1

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="New York, South Florida, California, Connecticut, Colorado, New Jersey, Texas and Massachusetts Real Estate | Douglas Elliman" name="Description"/>
<meta content="noodp" name="robots"/>
<script src="https://www.google.com/recaptcha/api.js"></script>
<link href="https://elliman.com/advanced-search" rel="canonical"/><title>New York, South Florida, California, Connecticut, Colorado, New Jersey, Texas and Massachusetts Real Estate | Douglas Elliman</title>
<script type="text/javascript">
<!--//--><![CDATA[//><!--
var _gaq = _gaq || [];
_gaq.push(['_setAccount', 'UA-2052622-4']);
_gaq.push(['_setDomainName', 'elliman.com']);
_gaq.push(['_setCustomVar', 4, 'LoggedInUser', 'No', 1]);
_gaq.push(['_trackPageview']);
_gaq.push(['_trackPageLoad

In [86]:
HEADER = 'https://www.elliman.com'
def get_apt_urls_per_page(soup):
    apartments = soup.find_all('li', class_='listing_address first')
    apt_urls = [apt.find('a')['href'] for apt in apartments]
    apt_urls = [f'{HEADER}{url}' for url in apt_urls]
    return apt_urls

In [87]:
get_apt_urls_per_page(soup_pg1)

['https://www.elliman.com/new-york-city/12-east-63rd-street-manhattan-qjnryze',
 'https://www.elliman.com/new-york-city/118-east-76th-street-manhattan-basrlqe',
 'https://www.elliman.com/new-york-city/10-east-76th-street-manhattan-iuqifyo',
 'https://www.elliman.com/new-york-city/11-st-lukes-place-manhattan-exnpepx',
 'https://www.elliman.com/new-york-city/12-east-80th-street-manhattan-djuusxd',
 'https://www.elliman.com/new-york-city/39-east-72nd-street-penthouse-manhattan-dwlnahy',
 'https://www.elliman.com/new-york-city/18-east-74th-street-manhattan-gyioxmw',
 'https://www.elliman.com/new-york-city/42-west-12th-street-manhattan-wokkpnq',
 'https://www.elliman.com/new-york-city/48-east-81st-street-manhattan-vmjymzz',
 'https://www.elliman.com/new-york-city/60-bethune-street-manhattan-syrzhui',
 'https://www.elliman.com/new-york-city/29-west-75th-street-manhattan-bsebich',
 'https://www.elliman.com/new-york-city/114-east-65th-street-manhattan-haugxns',
 'https://www.elliman.com/new-yo

In [90]:
page33 = get_webpage(33)
soup_pg33 = get_soup(page33)
apartments = soup_pg33.find_all('li', class_='listing_address first')
apt_urls = [apt.find('a')['href'] for apt in apartments]
apt_urls

['/new-york-city/4015-atlantic-avenue-brooklyn-dlhvhtu',
 '/new-york-city/110-38-68ave-queens-mrdklou',
 '/new-york-city/947-79th-street-brooklyn-gggurut',
 '/new-york-city/578-washington-avenue-brooklyn-trgzbzc',
 '/new-york-city/4720-grosvenor-avenue-bronx-agsnlju',
 '/new-york-city/43-bay-25th-street-brooklyn-oemfghs',
 '/new-york-city/322-graham-avenue-brooklyn-egvlsgs',
 '/new-york-city/144-15-south-drive-queens-ajxifbj',
 '/new-york-city/70-henry-street-b-brooklyn-kkkrjef',
 '/new-york-city/845-bay-ridge-avenue-brooklyn-rqbjftv',
 '/new-york-city/198-36-epsom-cors-queens-gaoosjj',
 '/new-york-city/145-huntington-street-3l-brooklyn-swimpdg',
 '/new-york-city/936-seneca-avenue-queens-ijidimq',
 '/new-york-city/31-boulevard-queens-wzkukpx',
 '/new-york-city/1960-west----4-street-brooklyn-avecrsw',
 '/new-york-city/5009-5th-street-queens-hfazsot',
 '/new-york-city/1949-east---12-street-brooklyn-dvpxgws',
 '/new-york-city/1449-east-28-street-brooklyn-vqcrzqx',
 '/new-york-city/150-jor

In [92]:
page500 = get_webpage(410)
soup_pg500 = get_soup(page500)
apartments = soup_pg500.find_all('li', class_='listing_address first')
apt_urls = [apt.find('a')['href'] for apt in apartments]
apt_urls

['/new-york-city/east-new-york-brooklyn-nllutnj',
 '/new-york-city/932-east---32-street-brooklyn-yxaigjg',
 '/new-york-city/406-east---58-street-brooklyn-ihdgfuh',
 '/new-york-city/118-42-154-street-queens-wrfsqlm',
 '/new-york-city/117-24-199-street-queens-uqpjrjr',
 '/new-york-city/115-23-123-street-queens-pfllzbj',
 '/new-york-city/111-43-130-street-queens-jvdbjwm',
 '/new-york-city/1312-needham-avenue-bronx-cxoidyp',
 '/new-york-city/85-13-60-avenue-queens-orbmqar',
 '/new-york-city/90-21-197-street-queens-drgmxlv',
 '/new-york-city/213-15-nashville-boulevard-queens-enpyhvu',
 '/new-york-city/604-city-island-avenue-bronx-omethrg',
 '/new-york-city/25-51-99th-street-queens-nhahubp',
 '/new-york-city/958-east-224th-street-bronx-imsgmfk',
 '/new-york-city/far-rockaway-queens-icrmiok',
 '/new-york-city/115-93-217th-street-queens-zfnokuq',
 '/new-york-city/933-east-88-street-brooklyn-qwwogvr',
 '/new-york-city/85-55-76-street-queens-zrtaoov',
 '/new-york-city/91-03-212-street-queens-huc

In [146]:
def get_apt_urls_ensemble(verbose=False, test=False):
    pg_num = 1
    stop = False
    apt_urls = []
    
    while not stop:
        
        time.sleep(2)
        
        if test and pg_num == 10:
            break
        
        if pg_num%20 == 0:
            if verbose:
                print('20 pages scraped, sleep 15 seconds')
            time.sleep(15)
            
        webpage = get_webpage(pg_num)
        soup_pg = get_soup(webpage)
        apt_urls_pg = get_apt_urls_per_page(soup_pg)
        more_listings = soup_pg.find('div', class_='_grid33 _alpha')
        if (not apt_urls_pg) and (not more_listings):
            stop = True
        else:
            apt_urls += apt_urls_pg
            if verbose:
                print(f'apartment URLs in page {pg_num} all scraped')
            pg_num += 1
    
    return apt_urls

In [ ]:
all_apt_urls = get_apt_urls_ensemble(verbose=True)

apartment URLs in page 1 all scraped
apartment URLs in page 2 all scraped
apartment URLs in page 3 all scraped
apartment URLs in page 4 all scraped
apartment URLs in page 5 all scraped
apartment URLs in page 6 all scraped
apartment URLs in page 7 all scraped
apartment URLs in page 8 all scraped
apartment URLs in page 9 all scraped
apartment URLs in page 10 all scraped
apartment URLs in page 11 all scraped
apartment URLs in page 12 all scraped
apartment URLs in page 13 all scraped
apartment URLs in page 14 all scraped
apartment URLs in page 15 all scraped
apartment URLs in page 16 all scraped
apartment URLs in page 17 all scraped
apartment URLs in page 18 all scraped
apartment URLs in page 19 all scraped
20 pages scraped, sleep 15 seconds
apartment URLs in page 20 all scraped
apartment URLs in page 21 all scraped
apartment URLs in page 22 all scraped
apartment URLs in page 23 all scraped
apartment URLs in page 24 all scraped
apartment URLs in page 25 all scraped
apartment URLs in page 2

In [141]:
page500 = get_webpage(209)
soup_pg500 = get_soup(page500)
apartments = soup_pg500.find_all('li', class_='listing_address first')
apt_urls = [apt.find('a')['href'] for apt in apartments]
apt_urls

['/new-york-city/163-03-92nd-street-queens-vperbot',
 '/new-york-city/11611-125th-street-queens-ioxezgf',
 '/new-york-city/82-65-88th-lane-queens-qkqajpw',
 '/new-york-city/8904-163rd-avenue-queens-duoadzx',
 '/new-york-city/2417-east-69-street-brooklyn-nvbiyng',
 '/new-york-city/2411-east-69-street-brooklyn-wpuetha',
 '/new-york-city/2234-westervelt-avenue-bronx-mhdcodw',
 '/new-york-city/25518-francis-lewis-boulevard-queens-bjmnulq',
 '/new-york-city/88-16-rutledge-avenue-queens-wafkewx',
 '/new-york-city/145-52-223rd-street-queens-esmojqn',
 '/new-york-city/1048-brooklyn-avenue-brooklyn-omkmere',
 '/new-york-city/18-danforth-street-brooklyn-hhnmiis',
 '/new-york-city/215-east-35th-street-street-brooklyn-yvbqtzo',
 '/new-york-city/227-07-56-avenue-queens-aanbidg',
 '/new-york-city/9-23-126-street-queens-mnrkgrq',
 '/new-york-city/86-77-musket-street-queens-qcevspg',
 '/new-york-city/215-east-35th-street-street-brooklyn-asudofu',
 '/new-york-city/150-12-25-drive-queens-yjysthw',
 '/ne

In [128]:
time.sleep(10)